# SQL & 探索的データ解析

この演習では，食料品即日配達サービス[Instacart](https://www.instacart.com)の注文データを用いる．演習を開始する前に，[データダウンロードリンク](https://www.instacart.com/datasets/grocery-shopping-2017)よりデータが納められたzipファイル（約200MB）をダウンロードしておくこと．

ダウンロードしたzipファイルには下記6つのcsvファイルが圧縮されている．
* ``departments.csv``: 商品売り場（カテゴリ）に関するデータ（21件）
* ``aisles.csv``: 商品棚（サブカテゴリ）に関するデータ（134件）
* ``products.csv``: 商品に関するデータ（約5万件）
* ``orders.csv``: 注文に関するデータ（約20万6000ユーザによる約340万件の注文）
* ``order_products__train.csv``: 各Instacartユーザが最も最近行った注文の明細データ
* ``order_products__prior.csv``: 各Instacartユーザが過去に行った注文の明細データ（最も最近の注文を除く）

各csvファイルの詳細は，``data/instacart``ディレクトリの``data-recipe.md``ファイルで確認することができる．

### Q1. データベース作成
ダウンロードしたcsvファイル内のデータをSQLiteのデータベースに格納せよ．テーブル名は``department``のように，csvファイル名から拡張子を取り除いたものにせよ．ただし，``order_products__train.csv``と``order_products__prior.csv``のデータについては，``order_products``というテーブルを作成し，そこにデータを格納せよ．カラム情報は``data/instacart``ディレクトリの``data-recipe.md``ファイルに記した内容に従うこと．

なお，必要であれば，``data/instacart``ディレクトリの``init_sqlite.sql``ファイルを利用してもよい．

### Q2. pandasデータフレームへの読み込み
``pandas``ライブラリの``read_sql``関数を用いると，SQLite上にあるデータベースにSQLを発行し，その結果を``pandas.DataFrame``データフレームに読み出すことができる．下記コードは，SQLite上にある``instacart``データベースの``department``テーブルの内容をデータフレームに読み込む例である．

``products``テーブル，``aisles``テーブル，``departments``テーブル，``orders``テーブル，``order_products``テーブルのレコード数を調べるSQL文を発行し，各テーブルのレコード数を表示せよ．

In [1]:
import sqlite3
import pandas as pd
import pandas.io.sql as psql

In [2]:
db = sqlite3.connect('../data/instacart/instacart.db') # DBと接続する
sql = "SELECT * FROM departments;"
df = psql.read_sql(sql, db)
db.close() # DBとの接続を閉じる（閉じないと開きっぱなしになるので注意）

# 最初の5件を表示
df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


### Q3. マスターデータ作成（テーブル結合 & 集約演算）
商品部門別および商品棚別に商品数を求めるSQL文を発行し，その結果をデータフレーム形式で表示せよ．なお，表示するデータフレームは下記の形式とせよ（ALLは特定のdepartmentにある全aisleを意味する）．また表示する結果は，商品数の降順でソートしたものとせよ．

| |department |aisle |iterm_count|
|---: |---:|---:|---:|
|0 |personal care |ALL |6563 |
|1 |snacks |ALL |6264 |
|... |... |... |... |
|... |personal care |vitamins supplements |1038 |
|...|personal care |hair care |816 |
|... |... |... |... |

### Q4. 注文行動に関する基礎情報（集約演算2）
ユーザID毎に注文回数（``order_frequency``）および注文の平均間隔（``avg_order_interval``）を把握したい．注文回数が10回以上99以下のユーザに関して，ユーザID毎の注文回数と注文平均間隔を求めるSQL文を発行し，注文平均間隔の上位10件（昇順）のユーザ情報のみ表示せよ．結果はデータフレーム形式で表示せよ．なお，ある注文が初回注文だった場合，前回の注文からの経過日数間隔はゼロと考えよ．

### Q5. 注文タイミング（ヒートマップによる可視化）
注文があった曜日および時間帯別（例: 土曜日の23時）の注文総数を集計するSQL文を発行し，その結果をヒートマップ形式で表示せよ．

### Q6. 注文間隔（棒グラフによる可視化）
前回の注文からの経過日数別に注文数を集計するSQL文を発行し，その結果を棒グラフで示せ．

### Q7. ユーザの利用頻度（副問い合わせ）
顧客の注文回数を集計することで注文回数別の顧客数を求めるSQL文を発行し，その結果を棒グラフで表示せよ．

### Q8. 再注文率（SQL上での算術演算）
商品売り場別の商品の注文総数および再注文率を計算するSQL文を発行し，その結果を表示せよ．なお，結果は再注文率でソートしたものを表示せよ．

### Q9. 商品棚にみる注文回数の分布（Window関数で四分位数を求める）
"pantry"売り場の商品棚別に注文回数の四分位数を求めるSQL文を発行し，その結果をデータフレームに格納し表示せよ．

### Q10. 商品棚別の注文数および再注文率（リッチな可視化）
商品棚別の商品の注文総数および再注文率を計算するSQL文を発行し，その結果をデータフレームに格納せよ．また，集計結果をもとに，X軸に注文総数（対数スケール），Y軸に再注文率とする散布図を描け．